#### Online Index Selection Via Combinatorial Contextual Multi Armed Bandits

In [1]:
import logging
import datetime
import os
import subprocess
import uuid

import numpy as np
import pyodbc
import sys
import random
import pandas as pd
import time
import os
from tqdm import tqdm
import logging
import re
import json
import itertools
import math
from collections import defaultdict
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

import IPython
notebook_path = IPython.get_ipython().starting_dir
target_subdirectory_path = os.path.abspath(os.path.join(os.path.dirname(notebook_path), 'database'))
sys.path.append(target_subdirectory_path)
from utils import *

from mab import *
from dta import *

In [2]:
# read workload queries from JSON file
def read_workload(workload_filepath):
    workload = []
    with open(workload_filepath) as f:
        line = f.readline()
        # read the queries from each line
        while line:
            workload.append(json.loads(line))
            line = f.readline()

    return workload

# Base directory containing the generated queries
workload_filepath = '../datagen/TPCH_workloads/TPCH_static_100_workload.json'

# Read the workload queries from file
workload = read_workload(workload_filepath)
print(len(workload))

2100


#### MAB index selection algorithm. On each round, do the following:

1) Generate candidate arms/indices using mini-workload from previous round
2) Generate context vector for each candidate arm
3) Select the best super-arm, i.e. configuration/subset of candidate indices
4) Materialize the super-arm configuration, then execute new mini-workload for current round


We will implement these 4 steps separately in the given order.

#### 1. Generation of Candidate indices

Test index generation for miniworkload of first 21 queries

In [3]:
mab = MAB()

All non-clustered indexes --> []
All nonclustered indexes removed.


In [3]:
connection = start_connection()

miniworkload = []
for query in workload[0:21]:
    # convert to Query object
    miniworkload.append(Query(connection, query['template_id'], query['query_string'], query['payload'], query['predicates'], query['order_bys'], query['group_bys']))

close_connection(connection)

In [14]:
connection = start_connection()

# genete candidate indices
i=0
index_arms = mab.generate_candidate_indices(connection, miniworkload[i:i+1], verbose=False)
print(index_arms.keys())

close_connection(connection)

Generating candidate indices for 1 queries...


Processing queries: 100%|██████████| 1/1 [00:00<00:00, 540.64it/s]

Generated 371 candidate indices
dict_keys(['IX_lineitem_l_shipdate', 'IXN_lineitem_l_shipdate_l_di', 'IXN_lineitem_l_shipdate_l_li', 'IXN_lineitem_l_shipdate_l_qu', 'IXN_lineitem_l_shipdate_l_re', 'IXN_lineitem_l_shipdate_l_ta', 'IXN_lineitem_l_shipdate_l_ex', 'IXN_lineitem_l_shipdate_l_di_l_li', 'IXN_lineitem_l_shipdate_l_di_l_qu', 'IXN_lineitem_l_shipdate_l_di_l_re', 'IXN_lineitem_l_shipdate_l_di_l_ta', 'IXN_lineitem_l_shipdate_l_di_l_ex', 'IXN_lineitem_l_shipdate_l_li_l_qu', 'IXN_lineitem_l_shipdate_l_li_l_re', 'IXN_lineitem_l_shipdate_l_li_l_ta', 'IXN_lineitem_l_shipdate_l_li_l_ex', 'IXN_lineitem_l_shipdate_l_qu_l_re', 'IXN_lineitem_l_shipdate_l_qu_l_ta', 'IXN_lineitem_l_shipdate_l_qu_l_ex', 'IXN_lineitem_l_shipdate_l_re_l_ta', 'IXN_lineitem_l_shipdate_l_re_l_ex', 'IXN_lineitem_l_shipdate_l_ta_l_ex', 'IXN_lineitem_l_shipdate_l_di_l_li_l_qu', 'IXN_lineitem_l_shipdate_l_di_l_li_l_re', 'IXN_lineitem_l_shipdate_l_di_l_li_l_ta', 'IXN_lineitem_l_shipdate_l_di_l_li_l_ex', 'IXN_lineitem_l_

#### 2. Generation of Context Vectors for Each Arm/Index

The context vector of each index can be defined as a concatenation of two pieces:

* Columns Piece:  a vector with length equal to the total number of columns in the database. Each entry in this vector corresponds to one of the columns and contains the value $10^{-j}$ where $j$ is the position of that column in the index, provided that column is in the index, otherwise the value is zero. 

* Derived Context Piece: a vector of length 2, first component contains time stamp of last round when the index was used and second component is the size of the index relative to the entire database

In [15]:
connection = start_connection()

# test context vector generation
context_vectors = mab.generate_contexts(connection, index_arms)
print(context_vectors.shape)

close_connection(connection)

(371, 124)


#### 3: Generation of super-arm/ best configuration

To generate the best configurations:

* compute the estimated upper bound on expected reward from each index
* then solve the 0-1 knapsack problem to find the subset of indices which maximizes total expected reward upper bound while satisfying memory constraint

In [17]:
# test super arm selection
selected_indices = mab.select_best_configuration(context_vectors, index_arms, config_memory_budget_MB=5*1024, verbose=True)


Filtered upper bounds: {'IX_lineitem_l_shipdate': 0.0010893481810414496, 'IXN_lineitem_l_shipdate_l_li': 0.5857764017553204, 'IXN_lineitem_l_shipdate_l_re': 0.5857764017553204, 'IXN_lineitem_l_shipdate_l_li_l_re': 1.034876564504173, 'IX_lineitem_l_returnflag': 0.0, 'IXN_lineitem_l_returnflag_l_li': 0.5849596287698582, 'IX_lineitem_l_linestatus': 0.0, 'IXN_lineitem_l_linestatus_l_re': 0.5849596287698582, 'IX_lineitem_l_shipdate_l_returnflag': 0.008706588662745762, 'IXN_lineitem_l_shipdate_l_returnflag_l_li': 0.5912011628802578, 'IX_lineitem_l_shipdate_l_linestatus': 0.008706588662745762, 'IXN_lineitem_l_shipdate_l_linestatus_l_re': 0.5912011628802578, 'IX_lineitem_l_returnflag_l_shipdate': 0.008706588662745762, 'IXN_lineitem_l_returnflag_l_shipdate_l_li': 0.5912011628802578, 'IX_lineitem_l_returnflag_l_linestatus': 0.00755929702917113, 'IX_lineitem_l_linestatus_l_shipdate': 0.008706588662745762, 'IXN_lineitem_l_linestatus_l_shipdate_l_re': 0.5912011628802578, 'IX_lineitem_l_linestatus_l

#### 4. Meterialize super-arm configuration and execute mini-workload, observe rewards 

In [43]:
connection = start_connection()

total_execution_cost, creation_cost, index_rewards = mab.materialize_execute(connection, selected_indices, miniworkload, index_arms, verbose=False)

print(index_rewards)

close_connection(connection)


Total execution cost: 28.466
{'IX_supplier_s_suppkey_s_nationkey': [0.019999999999999997, -0.01], 'IX_customer_c_mktsegment': [0.032, -0.239], 'IX_supplier_s_nationkey': [0.0035, -0.013], 'IX_customer_c_nationkey': [0.031, -0.127], 'IX_customer_c_custkey_c_nationkey': [0.0155, -0.113], 'IX_supplier_s_nationkey_s_suppkey': [0.0035, -0.013], 'IX_customer_c_custkey': [-0.011000000000000003, -0.183], 'IX_supplier_s_comment': [0.002999999999999999, -0.042], 'IX_part_p_container': [0.006666666666666668, -0.172], 'IX_part_p_brand_p_partkey': [0.001666666666666668, -0.431], 'IX_part_p_size_p_partkey': [0.016, -0.126], 'IX_customer_c_phone': [-0.10350000000000001, -0.156], 'IX_customer_c_acctbal_c_custkey': [-0.01750000000000001, -0.105], 'IX_part_p_brand_p_size_p_partkey': [0, -0.382], 'IX_customer_c_acctbal': [0, -0.157], 'IX_customer_c_nationkey_c_custkey': [0, -0.255], 'IX_part_p_partkey_p_brand': [0, -0.144], 'IX_customer_c_phone_c_custkey': [0, -0.214], 'IX_part_p_partkey_p_brand_p_size':

In [44]:
mab.update_parameters(selected_indices, index_rewards, index_arms)


#### Full MAB rounds

In [6]:
mab = MAB(alpha_decay_rate=0.95)

All non-clustered indexes --> [('dbo', 'lineitem', 'IXN_lineitem_l_returnflag_l_shipdate_l_linestatus_l_ta_l_ex_l_di_l_qu')]
All nonclustered indexes removed.


In [7]:
# run rounds of MAB with the same batch of queries (first 3 queries in the workload)
for i in range(1, 21):
    mab.step_round(miniworkload, i, config_memory_budget_MB=5*1024, verbose=False)

Running MAB agent for round 1...
Identifying new query templates and updating statistics...
Number of new query templates added: 21
Number of queries of interest: 0
Materializing configuration and executing queries...
Num indexes added:0, Num indexes removed: 0 Index creation costs: {}
Observed Rewards for indexes: {}

Total execution cost: 173.028, Total index creation cost: 0

Index selection counts: {}

Round completed. Time taken for recommendation: 0:03:27.164178 seconds.

Running MAB agent for round 2...
Identifying new query templates and updating statistics...
Number of new query templates added: 0
Number of queries of interest: 21
Generating candidate indices...
Generating candidate indices for 21 queries...


Processing queries: 100%|██████████| 21/21 [00:00<00:00, 131.70it/s]


Generated 24142 candidate indices
Generating context vectors...
Selecting best configuration...
Theta - parameter vector: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
Table filtered indexes: defaultdict(<class 'list'>, {'lineitem': ['IXN_lineitem_l_returnflag_l_shipdate_l_linestatus_l_ta_l_ex_l_di_l_qu', 'IXN_lineitem_l_returnflag_l_linestatus_l_shipdate_l_ta_l_ex_l_di_l_qu', 'IXN_lineitem_l_shipdate_l_returnflag_l_linestatus_l_ta_l_ex_l_di_l_qu'], 'part': ['IXN_part_p_type_p_size_p_mf_p_pa', 'IXN_part_p_size_p_type_p_mf_p_pa', 'IXN_part_p_type_p_mf_p_pa'], 'partsupp': ['IXN_partsupp_ps_suppkey_ps_p_ps_s_ps_a', 'IXN_partsupp_ps_suppkey_ps_s_ps_a', 'IXN_part

Processing queries: 100%|██████████| 21/21 [00:00<00:00, 204.03it/s]

Generated 24142 candidate indices
Generating context vectors...


...

Processing queries: 100%|██████████| 21/21 [00:00<00:00, 205.34it/s]

Generated 24142 candidate indices
Generating context vectors...


Selecting best configuration...
Theta - parameter vector: 
[ 0.00000000e+00 -1.34438306e+02 -2.67851351e-01 -4.96521539e-02
  9.51497583e-02  4.17833921e-01  2.66000000e-01  1.16493275e-01
  2.14964549e-01  2.08353879e-01  0.00000000e+00  0.00000000e+00
  1.93774123e+00  5.28215252e+00 -1.85851777e+00  0.00000000e+00
  0.00000000e+00 -1.09946569e-01  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.80831068e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.80831068e-01 -3.68120388e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.09009743e-01
  0.00000000e+00  1.04809743e-01  0.00000000e+00  3.62386458e-01
  8.44205843e-02  0.00000000e+00  1.45657188e-01  1.63543024e-01
  9.39779560e-02  0.00000000e+00  1.60400000e-01  1.26164078e-01
  9.23874967e-02  0.00000000e+00  1.02969850e-01  3.47105059e-02
  0.00000000e+00 -5.89233457e-01 -4.61734243e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+0

Processing queries: 100%|██████████| 21/21 [00:00<00:00, 133.67it/s]


Generated 24142 candidate indices
Generating context vectors...
Selecting best configuration...
Theta - parameter vector: 
[ 0.00000000e+00 -1.34455397e+02 -2.70390752e-01 -5.40901739e-02
  9.45524075e-02  3.98966266e-01  2.66000000e-01  9.80846587e-02
  2.03033007e-01  2.10769575e-01  0.00000000e+00  0.00000000e+00
  1.95824985e+00  5.20596680e+00 -1.24529994e+00  0.00000000e+00
  0.00000000e+00 -1.28590776e-01  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.81088929e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.81088929e-01 -3.68214156e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.08947866e-01
  0.00000000e+00  1.04747866e-01  0.00000000e+00  3.53224553e-01
  8.99988026e-02  0.00000000e+00  1.46151738e-01  1.63978976e-01
  1.08265257e-01  0.00000000e+00  1.60400000e-01  1.35452164e-01
  9.45534912e-02  0.00000000e+00  1.08314287e-01  3.61352922e-02
  0.00000000e+00 -6.04661457e-01

#### Comparison with DTA recommendation

In [10]:
dta = DTA_recommender()

In [13]:
queries = [q.query_string for q in miniworkload[:1]]
indexes_to_add, indexes_to_remove = dta.recommend_indexes(queries,  max_memory_Mb=20*1024, max_time_minutes=20) 

All non-clustered indexes --> []
All nonclustered indexes removed.
Microsoft (R) SQL Server dta
Version 20.2.30.0
Copyright (c) Microsoft. All rights reserved.

Tuning session successfully created. Session ID is 81.

Time elapsed: 00:00:02            
Workload consumed:  100%, Estimated improvement:   59%                         
Total time used: 00:00:02                                


                                                                               
                                                                                
Tuning process finished.
Successfully saved output XML file: \\wsl.localhost\Ubuntu\home\tanzid\Code\DBMS\MAB\session_output_session_7368988f-9abd-4ae1-8344-5a52b2d6ab9c.xml.
Successfully generated recommendations script: \\wsl.localhost\Ubuntu\home\tanzid\Code\DBMS\MAB\recommendations_session_7368988f-9abd-4ae1-8344-5a52b2d6ab9c.sql.
DTA recommendation time --> 21.138886 seconds.


In [14]:
for index in indexes_to_add:
    print(index)

for index in indexes_to_remove:
    print(index)    
    
# convert DTA index dicts to MAB index arm ids
DTA_recommended_configuration = []
for index in indexes_to_add:
    #print(index)
    index_columns = index['index_columns']
    # remove ASC/DESC from column names
    index_columns = [re.sub(' ASC| DESC', '', col) for col in index_columns]
    index_id = get_index_id(index_columns, index['table_name'], index['include_columns'])
    DTA_recommended_configuration.append(index_id)


DTA_recommended_configuration  


{'index_name': '_dta_index_lineitem_6_965578478__K9_K10_K11_5_6_7_8', 'table_name': 'lineitem', 'index_columns': ['l_returnflag ASC', 'l_linestatus ASC', 'l_shipdate ASC'], 'include_columns': ['l_quantity', 'l_extendedprice', 'l_discount', 'l_tax']}


['IXN_lineitem_l_returnflag_l_linestatus_l_shipdate_l_qu_l_ex_l_di_l_ta']

In [19]:
mab.best_configuration

{'IX_lineitem_l_shipdate_l_orderkey': <mab.Index at 0x7f97f0003010>,
 'IX_orders_o_shippriority': <mab.Index at 0x7f97f0011f10>,
 'IX_orders_o_orderkey': <mab.Index at 0x7f97f0002510>,
 'IX_partsupp_ps_partkey_ps_suppkey': <mab.Index at 0x7f97f00c1610>,
 'IX_customer_c_mktsegment': <mab.Index at 0x7f97e82b3e50>,
 'IXN_orders_o_orderdate_o_sh': <mab.Index at 0x7f97f01c0410>,
 'IX_orders_o_orderdate_o_orderkey': <mab.Index at 0x7f97f0000a10>,
 'IX_orders_o_orderdate': <mab.Index at 0x7f97f00c23d0>,
 'IX_lineitem_l_commitdate': <mab.Index at 0x7f97f00035d0>,
 'IXN_part_p_partkey_p_mf': <mab.Index at 0x7f97f009ae90>,
 'IXN_part_p_type_p_pa': <mab.Index at 0x7f97f00c3150>,
 'IXN_partsupp_ps_suppkey_ps_s': <mab.Index at 0x7f97f00c3010>,
 'IX_supplier_s_suppkey_s_nationkey': <mab.Index at 0x7f97f005d590>,
 'IX_supplier_s_suppkey': <mab.Index at 0x7f97f00996d0>,
 'IX_lineitem_l_suppkey': <mab.Index at 0x7f97f0000dd0>,
 'IXN_part_p_size_p_mf_p_pa': <mab.Index at 0x7f97f00c1c90>,
 'IXN_part_p_si

In [94]:
# check intersection of MAB and DTA recommendations
intersection = set(DTA_recommended_configuration).intersection(set(mab.best_configuration.keys()))

intersection

set()

In [27]:
test_query = miniworkload[0]
print(f"Query: {test_query.query_string}")
print(f"Predicates: {test_query.predicates}")
print(f"Payload: {test_query.payload}")
print(f"Order by: {test_query.order_by}")
print(f"Group by: {test_query.group_by}")



Query: select
	l_returnflag,
	l_linestatus,
	sum(l_quantity) as sum_qty,
	sum(l_extendedprice) as sum_base_price,
	sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
	sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
	avg(l_quantity) as avg_qty,
	avg(l_extendedprice) as avg_price,
	avg(l_discount) as avg_disc,
	count(*) as count_order
from
	lineitem
where
	l_shipdate <= DATEADD(dd, -63, CAST('1998-12-01' AS date))
group by
	l_returnflag,
	l_linestatus
order by
	l_returnflag,
	l_linestatus
;
Predicates: {'lineitem': {'l_shipdate': 'r'}}
Payload: {'lineitem': ['l_returnflag', 'l_linestatus', 'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax']}
Order by: {'lineitem': [['l_returnflag', 'asc'], ['l_linestatus', 'asc']]}
Group by: {'lineitem': ['l_returnflag', 'l_linestatus']}
